In [13]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from sentence_transformers import SentenceTransformer
from pymilvus import (
    connections,
    Collection,
    CollectionSchema,
    FieldSchema,
    DataType,
    utility,
)

In [14]:
# ---------------------------
# 1. Milvus + Embedding Setup
# ---------------------------
COLLECTION_NAME = "employee_policies"

def setup_rag(collection_name=COLLECTION_NAME):
    """
    Create a Milvus collection, connect to it, and insert sample data.
    """

    # Connect to Milvus using alias 'rag_db'
    connections.connect("rag_db", host="localhost", port="19530")

    # Drop collection if exists (for clean setup)
    if utility.has_collection(collection_name, using="rag_db"):
        utility.drop_collection(collection_name, using="rag_db")

    # Define schema
    fields = [
        FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
        FieldSchema(name="content", dtype=DataType.VARCHAR, max_length=1024),
        FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=384),
    ]

    schema = CollectionSchema(fields, description="Employee policy documents")
    collection = Collection(collection_name, schema, using="rag_db")

    # Load embedding model
    model = SentenceTransformer("all-MiniLM-L6-v2")

    # Sample data
    documents = [
        "Employees are paid bi-weekly via direct deposit.",
        "Vacation accrual starts after 90 days of employment.",
        "Health benefits become active after the first month.",
        "Remote work is allowed up to 3 days per week.",
        "Performance reviews are conducted annually in December."
    ]

    embeddings = model.encode(documents).tolist()

    # Insert data into Milvus
    collection.insert([documents, embeddings])

    # Create index
    collection.create_index(
        "embedding",
        {"index_type": "IVF_FLAT", "metric_type": "COSINE", "params": {"nlist": 128}}
    )
    collection.load()

    print(f"RAG setup complete with {len(documents)} sample records in '{collection_name}'.")
    print( )
    return collection_name

In [15]:
# ---------------------------
# 2. Setup the Data in the Vector DB
# ---------------------------

setup_rag()
# Load collection
collection = Collection(COLLECTION_NAME, using="rag_db")
collection.load()

# Query all records (limit = small number to avoid large dumps)
results = collection.query(expr="id >= 0", output_fields=["id", "content"], limit=10)

print(f"Retrieved {len(results)} records from collection '{COLLECTION_NAME}':\n")
for r in results:
    print(f"ID: {r['id']} | Content: {r['content']}")

RAG setup complete with 5 sample records in 'employee_policies'.

Retrieved 5 records from collection 'employee_policies':

ID: 462660528097722417 | Content: Employees are paid bi-weekly via direct deposit.
ID: 462660528097722418 | Content: Vacation accrual starts after 90 days of employment.
ID: 462660528097722419 | Content: Health benefits become active after the first month.
ID: 462660528097722420 | Content: Remote work is allowed up to 3 days per week.
ID: 462660528097722421 | Content: Performance reviews are conducted annually in December.
